# Nodes setup

In this notebook we show how to run a Corda node using Corda libraries and Gradle.

## Simple Corda node

The docker container has three Corda jar libraries in the ``/opt/corda`` directory:

In [2]:
!ls /opt/corda

bin				      diagnostic-89271760daa0.log
certificates			      logs
corda-tools-network-bootstrapper.jar  node-89271760daa0.log
corda-webserver.jar		      nodes
corda.jar


The two libraries that are used in this example are:
- ``corda.jar`` - the main Corda jar,
- ``corda-tools-network-bootstrapper.jar`` - is used to generate the network parameters.

The network parameters are a maps and rules how to communicate between each node in the network. You can read more bout the network parameter at [https://docs.corda.net/network-map.html#network-parameters](https://docs.corda.net/network-map.html#network-parameters). To generate the network parameters file, we need to have some initial nodes configuration. In this example we use just one node. A node can be defined in a ``.conf`` file. The detailes on each parameter can be found at
[https://docs.corda.net/releases/release-M4.1/corda-configuration-files.html](https://docs.corda.net/releases/release-M4.1/corda-configuration-files.html). The ``node.conf`` is a set of basic parameters of our node like the database, p2p, rpc ports, the user and keystore details. The P2P part is used by the nodes to communicate between each. RPC can used to communication with our web application. An example of a node configuration is given below. Our network parameters are quite short as it only contain one node.

In [ ]:
%%writefile /opt/corda/nodes/node.conf
p2pAddress = "localhost:10002"
rpcSettings {
    address: "localhost:10003"
    adminAddress: "localhost:10004"
}
h2port = 11000
emailAddress = "karol@codete.com"
myLegalName = "O=Codete, L=Krakow, C=PL"
keyStorePassword = "cordacadevpass"
trustStorePassword = "trustpass"
devMode = true
rpcUsers= [
    {
        user=corda
        password=portal_password
        permissions=[
            ALL
        ]
    }
]

To generate the network parameters we need to specify the directory that is searched for node configurations. You do it as follows:

In [1]:
!cd  /opt/corda && java -jar corda-tools-network-bootstrapper.jar --dir /opt/corda/

Bootstrapping local test network in /opt/corda
Using corda.jar in root directory
Copying corda.jar into node directory nodes
Nodes found in the following sub-directories: [nodes]
Found the following CorDapps: [corda-webserver.jar]
Copying CorDapp JARs into node directories
Waiting for all nodes to generate their node-info files...
... still waiting. If this is taking longer than usual, check the node logs.
Distributing all node-info files to all nodes
Loading existing network parameters... none found
Gathering notary identities
Generating contract implementations whitelist
QUASAR WARNING: Quasar Java Agent isn't running. If you're using another instrumentation method you can ignore this message; otherwise, please refer to the Getting Started section in the Quasar documentation.
New NetworkParameters {
      minimumPlatformVersion=4
      notaries=[]
      maxMessageSize=10485760
      maxTransactionSize=524288000
      whitelistedContractImplementations {
        
      }
      eventHo

You can see that when running for the first time, there are no network parameters found. Also no notary node is found in the configuration. The tool found the node configuration in the ``nodes`` directory and one Cordapp.

To run the node you can execute the node as below. We set some basic parameters like the h2 location, config file used, and the base directory where Corda starts with. You can also run the command in a console and check the Corda CLI. Type ``help`` to get more options and ``bye`` to exit Corda.

In [6]:
!java -Djava.security.egd=file:/dev/./urandom -Dcapsule.jvm.args="${JVM_ARGS}" -Dcorda.dataSourceProperties.dataSource.url="jdbc:h2:file:"${PERSISTENCE_FOLDER}"/persistence;DB_CLOSE_ON_EXIT=FALSE;WRITE_DELAY=0;LOCK_TIMEOUT=10000" -jar /opt/corda/nodes/corda.jar --base-directory /opt/corda/nodes/ --config-file /opt/corda/nodes/node.conf ${CORDA_ARGS}


   ______               __
  / ____/     _________/ /___ _
 / /     __  / ___/ __  / __ `/         Change is inevitable, except
/ /___  /_/ / /  / /_/ / /_/ /          from a vending machine.
\____/     /_/   \__,_/\__,_/

--- Corda Open Source 4.1 (c11f6c1) -------------------------------------------------------------


Logs can be found in                    : /opt/corda/nodes/logs
! ATTENTION: This node is running in development mode!  This is not safe for production deployment.
Database connection url is              : jdbc:h2:tcp://localhost:11000/node
Advertised P2P messaging addresses      : localhost:10002
RPC connection address                  : localhost:10003
RPC admin connection address            : localhost:10004
Loaded 1 CorDapp(s)                     : CorDapp corda-webserver version Unknown by Unknown with licence Unknown
Node for "Bank of Breakfast Tea" started up and registered in 44.81 sec


Welcome to the Corda interactive shell.
Useful commands include 'help' to

## Gradle

Nodes can be also configured also using Gradle. Build a Corda application this the more convinient way to do. Let's take a Kotlin Cordapp example. It's already cloned in this container, in case you want to do it locally, you can use the command:
``git clone https://github.com/corda/cordapp-template-kotlin.git``.
This example is also well described in the README file. The most important part is the nodes configuration that is done in ``build.gradle`` configuration. We have divided this file into two cells below for easier understanding of the content.

In the first part we have some general information on the Cordapp like libaries versions, dependencies and repositories. In the ``allprojects`` section we set Kotlin version and set the Corda repositories. The last section is on dependencies. We can see that the workflows and contracts are an important part of this project. We will cover this part in more details in the next notebook.

In [1]:
%%writefile /home/codete/workshop/cordapp-template-kotlin/build.gradle
buildscript {
    ext {
        corda_release_group = 'net.corda'
        corda_release_version = '4.1'
        corda_gradle_plugins_version = '4.0.45'
        kotlin_version = '1.2.71'
        junit_version = '4.12'
        quasar_version = '0.7.10'
        spring_boot_version = '2.0.2.RELEASE'
        spring_boot_gradle_plugin_version = '2.0.2.RELEASE'
        slf4j_version = '1.7.25'
        log4j_version = '2.11.2'
        corda_platform_version = '4'.toInteger()
    }

    repositories {
        mavenLocal()
        mavenCentral()
        jcenter()
        maven { url 'https://software.r3.com/artifactory/corda' }    
    }

    dependencies {
        classpath "org.jetbrains.kotlin:kotlin-gradle-plugin:$kotlin_version"
        classpath "net.corda.plugins:cordapp:$corda_gradle_plugins_version"
        classpath "net.corda.plugins:cordformation:$corda_gradle_plugins_version"
        classpath "net.corda.plugins:quasar-utils:$corda_gradle_plugins_version"
        classpath "org.springframework.boot:spring-boot-gradle-plugin:$spring_boot_gradle_plugin_version"
    }
}

allprojects {
    apply plugin: 'kotlin'

    repositories {
        mavenLocal()
        jcenter()
        mavenCentral()
        maven { url 'https://software.r3.com/artifactory/corda' }
        maven { url 'https://jitpack.io' }
    }

    tasks.withType(org.jetbrains.kotlin.gradle.tasks.KotlinCompile) {
        kotlinOptions {
            languageVersion = "1.2"
            apiVersion = "1.2"
            jvmTarget = "1.8"
            javaParameters = true   // Useful for reflection.
        }
    }

    jar {
        // This makes the JAR's SHA-256 hash repeatable.
        preserveFileTimestamps = false
        reproducibleFileOrder = true
    }
}

apply plugin: 'net.corda.plugins.cordapp'
apply plugin: 'net.corda.plugins.cordformation'
apply plugin: 'net.corda.plugins.quasar-utils'

sourceSets {
    main {
        resources {
            srcDir rootProject.file("config/dev")
        }
    }
}

dependencies {
    // Corda dependencies.
    cordaCompile "$corda_release_group:corda-core:$corda_release_version"
    cordaCompile "$corda_release_group:corda-node-api:$corda_release_version"
    cordaRuntime "$corda_release_group:corda:$corda_release_version"

    // CorDapp dependencies.
    cordapp project(":workflows")
    cordapp project(":contracts")

    cordaCompile "org.apache.logging.log4j:log4j-slf4j-impl:${log4j_version}"
    cordaCompile "org.apache.logging.log4j:log4j-web:${log4j_version}"
    cordaCompile "org.slf4j:jul-to-slf4j:$slf4j_version"
}

Overwriting /home/codete/workshop/cordapp-template-kotlin/build.gradle


Workflows and contracts are also in the ``cordapp`` section, because we need to build both. The task ``deployNodes`` consist of three nodes configuration and a notary node configuration.

In [2]:
%%writefile -a /home/codete/workshop/cordapp-template-kotlin/build.gradle
cordapp {
    info {
        name "CorDapp Template"
        vendor "Corda Open Source"
        targetPlatformVersion corda_platform_version
        minimumPlatformVersion corda_platform_version
    }
}

task deployNodes(type: net.corda.plugins.Cordform, dependsOn: ['jar']) {
    nodeDefaults {
        projectCordapp {
            deploy = false
        }
        cordapp project(':contracts')
        cordapp project(':workflows')
    }
    node {
        name "O=Notary,L=London,C=GB"
        notary = [validating : false]
        p2pPort 10002
        rpcSettings {
            address("localhost:10003")
            adminAddress("localhost:10043")
        }
    }
    node {
        name "O=PartyA,L=London,C=GB"
        p2pPort 10005
        rpcSettings {
            address("localhost:10006")
            adminAddress("localhost:10046")
        }
        rpcUsers = [[ user: "user1", "password": "test", "permissions": ["ALL"]]]
    }
    node {
        name "O=PartyB,L=New York,C=US"
        p2pPort 10008
        rpcSettings {
            address("localhost:10009")
            adminAddress("localhost:10049")
        }
        rpcUsers = [[ user: "user1", "password": "test", "permissions": ["ALL"]]]
    }
}

Appending to /home/codete/workshop/cordapp-template-kotlin/build.gradle


To generate the nodes configuration we should execute the ``deployNodes`` task as follows:

In [ ]:
!cd /home/codete/workshop/cordapp-template-kotlin/ && ./gradlew deployNodes

This generates nodes configuration in the ``build/nodes`` directory. We see a separate directory for each node and a ``runnodes`` script. It runs each node. You can also execute the command below in a console and use the Corda CLI.

In [ ]:
!cd /home/codete/workshop/cordapp-template-kotlin/build/nodes/ && ./runnodes